In [1]:
import requests
from requests_futures.sessions import FuturesSession
from requests_futures import sessions
import json
import psycopg2
from psycopg2 import Error
import pandas.io.sql as psql
import pandas as pd
from datetime import datetime, timedelta

In [142]:
try:
    conn = psycopg2.connect(dbname='d6nl8uste5t3fk', user='igtnybsnftnmqu',
                            password='ea76fc1725bd520b7f79115893279cf4905791f85140bd7af0bf677ee8a4dc3f',
                            host='ec2-54-166-167-192.compute-1.amazonaws.com', port='5432', sslmode='require')
    cur = conn.cursor()

    print('Connected!')

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

Connected!


In [3]:
table = psql.read_sql('SELECT * FROM home_customer', conn)

In [4]:
def check(params):
    session = sessions.FuturesSession(max_workers=3)
    futures = [session.get(url="https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin", params=p) for p in  params]
    for f in futures:
        f.result()

    session.close()
    
    return futures

In [85]:
pins=table.pin.unique()
avail = [0 for i in range(len(pins))]

current = []
for pin in pins:
    current.append(table[table['pin']==pin]['flag2'].values[0])

for i in range(7):
    params=[]
    date = datetime.today() + timedelta(i)

    for pin in pins:
        params.append({'pincode': str(pin), 'date': str(date.strftime("%d-%m-%Y"))})

    responses = check(params) 


    for i in range(len(pins)):
        res = responses[i].result().json()
        
        if len(res)!=0:
            for center in res['centers']:
                for j in range(len(center['sessions'])):
                    if center['sessions'][j]['available_capacity'] > 0 and center['sessions'][j]['min_age_limit']==18:
                        avail[i] = 1

In [110]:
for i in range(len(pins)):
    if avail[i]!=current[i]:
        if avail[i]>current[i]:
            #when vaccine becomes available

            #send sms ki vaccine aa gayi hai
            cur.execute(f'UPDATE home_customer SET flag1=1, flag2=1 WHERE pin={pins[i]};')
            cur.commit()
            

        elif avail[i]<current[i]:
            #when vaccine becomes unavailable

            #change flag2 everywhere to 0
            cur.execute(f'UPDATE home_customer SET flag1=0, flag2=0 WHERE pin={pins[i]};')
            cur.commit()

        elif avail[i]=1:
            #send sms to all with flag2=1 and flag1=0
            cur.execute(f'UPDATE home_customer SET flag1=1 WHERE flag1=0 and pin={pins[i]};')

In [145]:
x=requests.post('http://2factor.in/API/V1/293832-67745-11e5-88de-5600000c6b13/ADDON_SERVICES/SEND/PSMS',
            data = {
                'From'  :   'ABCDEF',
                'Msg'   :   'Hello boi',
                'To'    :   '7588860475'
            }
            )

In [147]:
x.json()

{'Status': 'Error', 'Details': 'Invalid API Key'}